In [1]:
!wget -nc https://aplj13.ifap.min-agricultura.pt/idigital.consultarFundos//pub/FEAGA_FEADER_2014.zip

File 'FEAGA_FEADER_2014.zip' already there; not retrieving.



In [2]:
import zipfile
import pandas as pd

In [3]:
z = zipfile.ZipFile('FEAGA_FEADER_2014.zip')
df = pd.read_csv(z.open('FEAGA_FEADER_2014.csv'), delimiter=';', encoding='latin1', skiprows=1)
df.head()

,Exercício,Denominação,Distrito,Concelho,Código Postal,Medida,Montante
0,2014,""" O MORRO "" - FABRICAÇÃO DE QUEIJOS, LDA.",01,47,9900,IV/B.1.8,"34378,05"
1,2014,""" O MORRO "" - FABRICAÇÃO DE QUEIJOS, LDA.",01,47,9900,IV/B.4.3,"11480,35"
2,2014,""" O MORRO "" - FABRICAÇÃO DE QUEIJOS, LDA.",01,47,9900,Total,"45858,4"
3,2014,A & D SERVIÇOS E INVESTIMENTOS LDA,02,13,4640,III.6,"3211,56"
4,2014,A & D SERVIÇOS E INVESTIMENTOS LDA,02,13,4640,IV/B.1.6,"7002,61"


In [4]:
df = df.rename(columns={
        u'Exercício': 'year',
        'Medida': 'scheme',
        'Montante': 'amount',
        u'Denominação': 'recipient_name',
        u'Código Postal': 'recipient_postcode',
    })
df.head()

,year,recipient_name,Distrito,Concelho,recipient_postcode,scheme,amount
0,2014,""" O MORRO "" - FABRICAÇÃO DE QUEIJOS, LDA.",01,47,9900,IV/B.1.8,"34378,05"
1,2014,""" O MORRO "" - FABRICAÇÃO DE QUEIJOS, LDA.",01,47,9900,IV/B.4.3,"11480,35"
2,2014,""" O MORRO "" - FABRICAÇÃO DE QUEIJOS, LDA.",01,47,9900,Total,"45858,4"
3,2014,A & D SERVIÇOS E INVESTIMENTOS LDA,02,13,4640,III.6,"3211,56"
4,2014,A & D SERVIÇOS E INVESTIMENTOS LDA,02,13,4640,IV/B.1.6,"7002,61"


In [5]:
df['recipient_location'] = df.apply(lambda x: u'%s, %s' % (x['Concelho'], x['Distrito']), 1)
df['amount'] = pd.to_numeric(df['amount'].str.replace(',', '.'))
df.head()

,year,recipient_name,Distrito,Concelho,recipient_postcode,scheme,amount,recipient_location
0,2014,""" O MORRO "" - FABRICAÇÃO DE QUEIJOS, LDA.",01,47,9900,IV/B.1.8,34378.05,"47, 01"
1,2014,""" O MORRO "" - FABRICAÇÃO DE QUEIJOS, LDA.",01,47,9900,IV/B.4.3,11480.35,"47, 01"
2,2014,""" O MORRO "" - FABRICAÇÃO DE QUEIJOS, LDA.",01,47,9900,Total,45858.40,"47, 01"
3,2014,A & D SERVIÇOS E INVESTIMENTOS LDA,02,13,4640,III.6,3211.56,"13, 02"
4,2014,A & D SERVIÇOS E INVESTIMENTOS LDA,02,13,4640,IV/B.1.6,7002.61,"13, 02"


In [6]:
from slugify import slugify
df['recipient_id'] = df.apply(lambda x: 'PT-%s-%s' % (x['recipient_postcode'], slugify(x['recipient_name'])), 1)
df = df.drop(['Distrito', 'Concelho'], 1)
df['currency'] = 'EUR'
df['country'] = 'PT'
df.head()

,year,recipient_name,recipient_postcode,scheme,amount,recipient_location,recipient_id,currency,country
0,2014,""" O MORRO "" - FABRICAÇÃO DE QUEIJOS, LDA.",9900,IV/B.1.8,34378.05,"47, 01",PT-9900-o-morro-fabricacao-de-queijos-lda,EUR,PT
1,2014,""" O MORRO "" - FABRICAÇÃO DE QUEIJOS, LDA.",9900,IV/B.4.3,11480.35,"47, 01",PT-9900-o-morro-fabricacao-de-queijos-lda,EUR,PT
2,2014,""" O MORRO "" - FABRICAÇÃO DE QUEIJOS, LDA.",9900,Total,45858.40,"47, 01",PT-9900-o-morro-fabricacao-de-queijos-lda,EUR,PT
3,2014,A & D SERVIÇOS E INVESTIMENTOS LDA,4640,III.6,3211.56,"13, 02",PT-4640-a-d-servicos-e-investimentos-lda,EUR,PT
4,2014,A & D SERVIÇOS E INVESTIMENTOS LDA,4640,IV/B.1.6,7002.61,"13, 02",PT-4640-a-d-servicos-e-investimentos-lda,EUR,PT


In [7]:
df['year'].value_counts()

2014    654359
Name: year, dtype: int64

In [8]:
df.to_csv('pt_2014.csv', index=False, encoding='utf-8')